# Read LD matrix

In [1]:
import numpy as np

In [2]:
files = [
    '../phase2/ld/eas.eur.22.0.ld.bin',
    '../phase2/ld/eas.eur.22.1.ld.bin' ]

In [3]:
Sigmas = np.array([ np.fromfile(f, 'double', -1, '') for f in files ])

In [4]:
M = np.sqrt(Sigmas.shape[1])
if (M != int(M)):
    print('Not a square matrix')
M = int(M)

In [5]:
Sigmas = Sigmas.reshape((2,M,M))

# Massage LD matrix

In [6]:
l = 0.1
L = np.diag(np.full(M, l))

In [7]:
Sigma_primes = Sigmas + np.array([L,L])

# Generate genotypes

In [8]:
import numpy.random as nr

In [9]:
N = 100

In [10]:
Xs = np.array([nr.multivariate_normal(np.zeros(M), Sigma, N) for Sigma in Sigma_primes])

In [12]:
import scipy.stats as ss

In [17]:
Gs = ss.binom(2,0.5).isf(ss.norm(0,1+l).sf(Xs))

In [24]:
As = np.array([np.corrcoef(G.T) for G in Gs])

# Generate effect sizes

In [ ]:
Mc = 30

In [ ]:
rho_b = 0.8

In [ ]:
def corr_matrix(rho_b, M=2):
    a = (1-rho_b)*np.diag(np.ones(M))
    b = rho_b*np.ones_like(a)
    return(a+b)

In [ ]:
bs = nr.multivariate_normal(np.zeros(2), corr_matrix(rho_b), Mc)

In [ ]:
Is = nr.choice(M, Mc, False)
Is.sort()

In [ ]:
mask_causal = np.zeros(M, dtype='bool')
mask_causal[Is] = True
mask_tagged = ~ mask_causal

# Generate phenotypes

In [ ]:
Ys = np.array([X[:,mask_causal].dot(b) for X, b in zip(Xs, bs.T)])

# REML

In [ ]:
import statsmodels.api as sm

In [ ]:
Q = sm.regression.mixed_linear_model.MixedLM(
    Ys[0],              # endog
    np.ones((N,1)),     # exog
    np.ones(N),         # groups
    np.corrcoef(X[0])   # exog_re
)

In [ ]:
Q.fit()